In [24]:
#Bonjour monsieur, c'est steven. Si le fichier mets des erreur d'import lancé 
#ce que j'ai mis en commentaire
#Pour le lien : http://127.0.0.1:8050
#par manque de temps je n'ai pue mettre une partie des derniers graphes
#mais on peut cliquer sur les plot pour choisir quelle 'paramètres' afficher (carré sur la gauche des graphes)

! pip install dash==2.0.0

#python -m venv ~/.venv/dash
#source ~/.venv/dash/bin/activate
#python -m pip install --upgrade pip
#pip install jupyter-dash
#pip install jupyterlab
#jupyter-lab


import dash
from jupyter_dash import JupyterDash
from dash import dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Output,Input,State
import folium
import json
from urllib.request import urlopen

In [25]:
df = pd.read_csv('data.csv')
df = df.drop(['Unnamed: 0'],axis=1)
df["Date mutation"] = df["Date mutation"].astype('datetime64[ns]')
df["Code departement"] = df["Code departement"].astype(str)
df["Valeur fonciere"] = df["Valeur fonciere"].astype(str)
df["Valeur fonciere"] = df["Valeur fonciere"].apply(lambda x : float(x.replace(',','.')))
dm = df
dm.dropna(subset = ["Nature culture"], inplace=True)

In [26]:
fg = dm.groupby(['Code departement','Date mutation'])['Surface terrain'].sum()
fg = fg.reset_index()
fg
fig = px.bar(fg, x="Date mutation", y="Surface terrain", color='Code departement', orientation='v', height=600,
             title='Surface par date et le département', color_discrete_sequence = px.colors.cyclical.mygbm)

In [27]:
fig2 = px.treemap(fg.sort_values(by='Surface terrain' ,ascending=False).reset_index(drop=True), path=["Code departement"], values="Surface terrain", height=700,title='Surface/Departement',color_discrete_sequence = px.colors.qualitative.Prism)
fig2.data[0].textinfo = 'label+text+value'

In [28]:
fg = dm
fg.dropna(subset = ["Nature culture"], inplace=True)
fg["Valeur fonciere"] = fg["Valeur fonciere"].astype(str)
fg["Valeur fonciere"] = fg["Valeur fonciere"].apply(lambda x : float(x.replace(',','.')))
fg = fg.groupby(['Nature culture','Date mutation'])['Valeur fonciere'].sum()
fg = fg.reset_index()
fig3 = px.bar(fg, x="Date mutation", y="Valeur fonciere", color='Nature culture', orientation='v', height=600,
             title='Investiment de type nature culture par date', color_discrete_sequence = px.colors.cyclical.mygbm)

In [29]:
data = urlopen("https://france-geojson.gregoiredavid.fr/repo/departements.geojson").read()
geo = json.loads(data.decode('utf-8'))
fig4=folium.Map(location=[48.856578,2.351828])
table=dm.groupby(['Code departement'])['Nature mutation'].count()
fig4.choropleth(geo_data=geo, key_on="feature.properties.code", data=table,columns=['Code departement','Nature mutation']
                      ,fill_color="Spectral")
#fig4

In [30]:
app = JupyterDash(__name__)


app.layout = html.Div(children = [
    html.H1("Dashboard"),
    html.Div([
        dcc.Dropdown(
            id='dropdown1',
            options=[{'label': 'Surface vendu par date et département', 'value': '1'},
                    {'label': 'La surface des département','value': '2'},
                    {'label': 'Vente de terrain type culture selon la date','value': '3'},]
            ,value = '1',)
        ,],),

    html.Div(id='tablecontainer1'),
    html.Div(
        dcc.Graph(
            id='graph1',
            className='dropgraph',
        ),
    )],)

#{'label': 'Carte de la france selon le nombre de vente liée à la nature culture','value': '4'},

@app.callback(
Output('graph1', 'figure'), 
[Input('dropdown1', 'value')]
)
#graph plot and styling
def update_graph(value):
    if value == '1':
        return fig
    elif value == '2':
        return fig2
    elif value == '3':
        return fig3
    
#elif value == '4':
        #return fig4

app.run_server(mode="jupyterlab")

In [31]:
#http://127.0.0.1:8050